Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 0 ns, sys: 5.14 ms, total: 5.14 ms
Wall time: 12.3 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.20.1.60/10407/1 Dashboard: http://10.20.1.60:8787/status,Cluster Workers: 1 Cores: 6 Memory: 8.36 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 229 ms, sys: 31.5 ms, total: 260 ms
Wall time: 1.99 s


[1.0235089319398893,
 1.8984162949564887,
 1.7599158860553437,
 1.8707183841110906,
 1.9789707306815245,
 1.6316531780584536,
 1.1703776237091148,
 1.0320421555451391,
 1.0802415331257924,
 1.0201799136016514,
 1.4228759244961005,
 1.1624436023287978,
 1.7640841054760197,
 1.666774322515694,
 1.5010090125013096,
 1.4411419085859505,
 1.1035550923168098,
 1.103045303795509,
 1.1659120224076451,
 1.8236281223112205,
 1.4084010926097439,
 1.547617390362912,
 1.3368059921299151,
 1.3381618404877758,
 1.378325324249119,
 1.4677451521224256,
 1.6422282663402101,
 1.4386257281153245,
 1.3772493429618065,
 1.8659430038381246,
 1.6666900612535955,
 1.716373886160779,
 1.2034551393795447,
 1.4388924329702997,
 1.7513550203469674,
 1.6090093424966854,
 1.9795254884846423,
 1.2428746779840458,
 1.7362336108154672,
 1.2728870645405126,
 1.4322307787069077,
 1.3365943406223333,
 1.1562441630076248,
 1.8805134344981858]

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.4575736649082542,
 1.6415384609003794,
 1.8399006904905906,
 1.6253841519175785,
 1.146973176975918,
 1.313591005014624,
 1.5215828971302614,
 1.8392265098633938,
 1.2348080771003627,
 1.3050740010879731,
 1.7672363850615636,
 1.0791185987496124,
 1.9486456642146428,
 1.9794009017198337,
 1.7909642414241707,
 1.9829119280788121,
 1.6980650278082883,
 1.336346124656528,
 1.7964688561440953,
 1.8250315164613133,
 1.9422476907836965,
 1.2112577906173798,
 1.3025351680972297,
 1.280025002283288,
 1.8531248463544996,
 1.8351044759315476,
 1.4704617662273156,
 1.598365630627944,
 1.7414806874629707,
 1.4678774134211645,
 1.5145552817910335,
 1.5446060169611822,
 1.8129627736938345,
 1.929570686351883,
 1.835058288386898,
 1.0692269491505193,
 1.6448340643958486,
 1.8674253373512888,
 1.966130226684733,
 1.8008560409997962,
 1.2921374149710587,
 1.381147612205647,
 1.8978838433779703,
 1.2821648095661762,
 1.0810829692955717,
 1.6593815568309582,
 1.2072684851515132,
 1.2969108300906833,
 

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 209 ms, sys: 27.6 ms, total: 236 ms
Wall time: 2.11 s
